# 出站认证 (Outbound Auth)

出站认证允许agent 和 AgentKit Gateway代表在入站认证期间已经过身份验证和授权的用户安全地访问火山引擎资源和第三方服务。要将授权与火山引擎资源或第三方服务集成，需要同时配置入站认证和出站认证。

通过 Agent Identity 支持的最小权限访问和安全权限委托，agent可以无缝且安全地访问火山引擎资源和第三方工具，如 飞书、Coze、GitHub 和 Google。agent可以代表用户或独立地在这些服务上执行操作，前提是有预先授权的用户同意。此外，您可以使用安全令牌库减少用户授权疲劳，并创建流畅的 AI Agent 体验。

## 核心概念

### 出站认证配置

首先，您需要向第三方提供商注册您的客户端应用程序，然后创建出站认证。您需要指定如何验证对火山引擎资源或第三方服务或 AgentKit Gateway 目标的访问。您可以使用 OAuth 2LO/3LO 或 API 密钥。对于 OAuth，您可以从 Agent Identity 提供的提供商中选择。在这种情况下，您需要输入来自 Agent Identity 的提供商配置详细信息。或者，您可以提供自定义提供商的详细信息。

当用户想要访问火山引擎资源或第三方服务或 AgentKit Gateway 目标时，出站认证会确认入站认证提供的访问令牌是否有效，如果有效，则允许访问资源。

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>

### 资源凭据提供商

这是agent代码用来检索下游资源服务器（例如 Google、GitHub）凭据以访问它们的组件，例如从 Gmail 获取电子邮件、向 Google 日历添加会议。它消除了agent开发人员在最终用户、agent代码和外部授权服务器之间实现 2LO 和 3LO OAuth2 编排流程的繁重工作。Agent Identity 提供自定义 OAuth2 凭据提供商和内置提供商列表，如 飞书、Coze、GitHub以及火山引擎各产品服务，其中授权服务器端点和提供商特定参数已预填充。
  

Agent Identity 为Agent开发人员提供 OAuth2 和 API 密钥凭据提供商，以便与支持 OAuth2 或 API 密钥的外部资源进行身份验证。在以下示例中，我们将引导您配置 OAuth2 凭据提供商。然后agent可以使用 OAuth2 凭据提供商为任何agent操作检索访问令牌。请参阅文档了解其他凭据提供商。

#### 创建资源凭据提供商

以下是创建 OAuth2 资源凭据提供商的示例。在此示例中，我们为 "Lark" 创建凭据提供商。这是自定义资源凭据提供商。您也可以使用 Agent Identity 控制台配置此凭据提供商。

```
from veadk.integrations.ve_identity import IdentityClient

identity_client = IdentityClient(region="cn-beijing")

# 配置 Lark OAuth2 提供商 - 代表用户
lark_provider = identity_client.create_oauth2_credential_provider({
    "name":"<您的凭据提供商名称>",
    "vendor":4,  # Lark Vendor
    "config":{
        "ClientId": "<your-lark-client-id>",
        "ClientSecret": "<your-lark-client-secret>",
        "Scopes": [
            "docs:document.content:read",
            "docx:document:readonly",
        ],
        "RedirectUrl": "https://cis-test.cn-beijing.volces.com/api/v1/oauth2callback",
    },
    "pool_name": "<您的agent身份池名称>"
})
print(lark_provider)

```

#### 使用资源凭据提供商检索 OAuth2 访问令牌

以下是从凭据提供商检索 OAuth2 访问令牌的示例。然后agent可以使用访问令牌与受 OAuth 保护的资源进行交互。例如：在我们的示例中，我们通过Lark API访问飞书文档，并且完成对于文档内容的查询。为了从凭据提供商检索凭据（如 access_token），您可以按如下所示封装您的函数。

```
import asyncio
from veadk.integrations.ve_identity.function_tool import OAuth2FunctionTool

async def need_token_3LO_async(*, access_token: str):
    print(access_token)


need_token_3LO_tool = OAuth2FunctionTool(
    func=need_token_3LO_async,
    provider_name="<您的凭据提供商名称>",
    auth_flow="USER_FEDERATION", # 代表用户 (3LO) 流程
    force_authentication=True,
)

await need_token_3LO_tool.run_async(args=[], tool_context=None)
```

以下是您可以与 OAuth2FunctionTool 包装类一起使用的各种参数。


| 参数名称             | 描述                                                              |
|:--------------------|:-------------------------------------------------------------------------|
| provider_name       | 凭据提供商名称                                             |
| into                | 要注入令牌的参数名称                                  |
| scopes              | 要请求的 OAuth2 范围                                                 |
| auth_flow           | 认证流程类型 ("M2M" 或 "USER_FEDERATION")                    |
| callback_url        | OAuth2 回调 URL                                                      |
| force_authentication| 强制重新认证                                                  |
| identity_client              | 可选参数，用于传入初始化好的Agent Identity Client                                                 |
| region           | 可选参数，用于指定Agent Identity服务的区域，默认为cn-beijing，在未提供identity_client的情况下生效                    |

## 概述

在本教程中，我们将使用 AgentKit SDK 开发一个文档查询agent，该agent可以访问用户飞书文档中的内容。我们将配置凭据提供商来帮助管理与 Lark 的凭据。我们可以使用自定义的 OAuth2 提供商并修改我们的代理代码来调用凭据提供商，并使用 access_token 从 Lark API 获取用户的文档内容。

**说明**：你可以在[使用 OAuth2 USER_FEDERATION 进行出站认证](https://volcengine.github.io/veadk-python/auth/oauth2-user-federation-outbound/)找到更详细的文档。

### Tutorial 架构

<div style="text-align:center">
    <img src="images/outbound_auth_3lo.svg" width="90%"/>
</div>


### Tutorial 说明

| 信息                | 详情                                                                  |
|:--------------------|:-------------------------------------------------------------------------|
| Agent类型            | 单Agent                                                                   |
| Agent框架            | Volcengine Agent Development Kit(VeADK) Agents                                                           |
| Tutorial 核心模块            | AgentKit Runtime, Agent Identity |
| 教程复杂度          | 中等                                                                   |
| 使用的 SDK          | AgentKit Python SDK, VeADK, volcengine-python-sdk                             |
| 凭据提供商          | 类型：OAuth2 - Lark 自定义提供商                                          |


### 教程关键特性

* 在 AgentKit Runtime 上托管Agent
* 使用火山方舟模型
* 使用 AgentKit Python SDK
* 使用 Agent Identity 出站认证与 OAuth2 Lark 密钥凭据提供商
* 支持飞书文档内容查询和分析

## 教程

### 先决条件

要执行本教程，您需要：
* Python 3.10+
* Volcengine 凭据
* VeADK
* [uv](https://docs.astral.sh/uv/#scripts)
* [Lark企业应用](https://open.larkoffice.com/app)

In [ ]:
!uv pip install --force-reinstall -U -r requirements.txt -q # 您可以通过 `--index` 参数指定镜像源以加快安装速度。

设置火山引擎 AccessKey 和 SecretKey 来使用 Agent Identity 接口以及进行 AgentKit Runtime 部署

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
from agentkit.toolkit import sdk

# 检查 .env 文件是否存在
env_file = Path('.env')
env_template = Path('.env.template')

if not env_file.exists():
    if env_template.exists():
        raise FileNotFoundError(
            "未找到 .env 文件！\n"
            "请先复制模板文件：cp .env.template .env\n"
            "然后填写您的配置信息。"
        )
    else:
        raise FileNotFoundError(
            "未找到 .env 文件！请创建 .env 文件并设置以下环境变量：\n"
            "- VOLCENGINE_ACCESS_KEY\n"
            "- VOLCENGINE_SECRET_KEY\n"
            "- MODEL_API_KEY"
        )
    
load_dotenv()

VOLCENGINE_ACCESS_KEY = os.getenv("VOLCENGINE_ACCESS_KEY")
VOLCENGINE_SECRET_KEY = os.getenv("VOLCENGINE_SECRET_KEY")
MODEL_API_KEY = os.getenv("MODEL_API_KEY")
LARK_APP_ID = os.getenv("LARK_APP_ID")
LARK_APP_SECRET = os.getenv("LARK_APP_SECRET")

if not VOLCENGINE_ACCESS_KEY:
    raise ValueError("请设置 VOLCENGINE_ACCESS_KEY环境变量")

if not VOLCENGINE_SECRET_KEY:
    raise ValueError("请设置 VOLCENGINE_ACCESS_KEY环境变量")

if not MODEL_API_KEY:
    raise ValueError("请设置 MODEL_API_KEY环境变量")

### 给Lark创建 OAuth2.0 APP（代表用户流程）
按照以下步骤注册您的应用、创建项目并配置 Lark 文档访问的 OAuth 凭据：

在本节中，我们将配置您的 Lark 应用以访问具有文档读取权限的 Lark Document API。

#### 步骤一：创建飞书自建应用

1. **创建应用**
   1. 登录 [飞书开放平台开发者后台](https://open.larkoffice.com/app?lang=zh-CN)
   2. 点击 **创建企业自建应用**
   3. 填写应用名称和描述
   4. 创建成功后，进入应用详情页

2. **获取应用凭据**
   1. 在 **凭证与基础信息** 功能页获取应用的 **App ID** 和 **App Secret**
   2. 复制这些凭据以供后续配置使用

#### 步骤二：配置OAuth重定向URL

1. 在应用详情页，选择 **安全设置** 一栏
2. 在右侧的 **重定向URL** 中添加接收授权码的页面地址
   
   **重要：请根据您所在的区域更新 URL。**
   
   对于 cn-beijing，重定向 URI 是：
   ```
   https://auth.id.cn-beijing.volces.com/api/v1/oauth2callback
   ```

#### 步骤三：申请API权限

1. 在 **开发配置 > 权限管理** 功能页，为应用申请 API 权限
2. 点击 **开通权限**，在右侧弹出的界面内选择 **用户身份权限**
3. 搜索并申请以下权限：
   - **查看、评论、编辑和管理云空间中所有文件 (drive:drive)** - 用于访问文档内容
   - **获取用户身份信息 (authen:user.info)** - 用于OAuth登录流程
4. 根据需要，还可以申请：
   - **查看、评论和编辑文档 (docs:document.content:read)** - 文档内容读取
   - **查看文档 (docx:document:readonly)** - 新版文档只读访问
5. 点击 **确认开通权限**

#### 步骤四：发布应用（如需要）

如果申请的权限需要审核，则必须发布应用使配置生效：

1. 在应用详情页左侧进入 **应用发布 > 版本管理与发布** 功能页
2. 点击 **创建版本**
3. 在 **版本详情** 页面，配置应用的版本号、能力、更新说明、可用范围等信息
4. 点击 **保存** 并 **确认发布**
5. 等待企业管理员审核通过后，API 权限正式生效

#### OAuth 2.0 流程说明

飞书的网页应用登录基于 OAuth 2.0 标准协议实现，整体流程如下：

1. **获取授权码 (code)** - 用户授权后获取临时授权码
2. **获取 user_access_token** - 使用授权码换取用户访问令牌
3. **获取用户身份信息** - 验证用户身份
4. **刷新 user_access_token** - 令牌过期时进行刷新

#### 在您的Agent中使用凭据

在下一节中，我们将配置资源凭据提供商，使用获取的 App ID、App Secret 和重定向 URI 进行 OAuth 3-legged 流程，让Agent能够代表用户访问飞书文档。

### 配置 Lark OAuth2 凭据提供商

修改以下代码并替换以下内容：
1. "your-lark-app-id" 替换为上述步骤中记录的"App ID"
2. "your-lark-app-secret" 替换为上述步骤中记录的"App Secret"

更新 App ID 和 App Secret 后，运行以下代码为 Lark 创建凭据提供商。

Agent Identity 中的资源凭据提供商充当智能中介，管理代理、身份提供商和资源服务器之间的复杂关系。每个提供商封装特定服务或身份系统所需的特定端点配置。该服务为流行服务提供内置提供商，包括 Google、GitHub等，其中授权服务器端点和提供商特定参数已预配置以减少开发工作。Agent Identity 通过可配置的 OAuth2 凭据提供商支持自定义配置，可以定制以与任何 OAuth2 兼容的资源服务器一起工作。

In [ ]:
from veadk.integrations.ve_identity import IdentityClient
from volcenginesdkcore.rest import ApiException

# 配置 Lark OAuth2 凭证提供者
identity_client = IdentityClient(region="cn-beijing")
try:
    lark_provider = identity_client.create_oauth2_credential_provider(
        {
            "name": "lark-user-oauth-provider",
            "vendor": 4,  # Lark Vendor
            "config": {
                "ClientId": LARK_APP_ID if LARK_APP_ID else "your-lark-app-id",
                "ClientSecret": LARK_APP_SECRET if LARK_APP_SECRET else "your-lark-app-secret",
                "Scopes": [
                    "docs:document.content:read",
                    "docx:document:readonly",
                ],
                "RedirectUrl": "https://auth.id.cn-beijing.volces.com/api/v1/oauth2callback",  # 使用Agent Identity提供的回调地址
            },
        }
    )

    print(lark_provider)
except ApiException as e:
    if "Duplicate entry" in e.reason:
        print("lark-user-oauth-provider already exists")
    else:
        print(e.reason)

### 准备您的 Agent 以部署到 AgentKit Runtime 并使用资源凭据提供商

现在让我们将 Agent 部署到 AgentKit Runtime。为此我们需要：
* 导入 Runtime App：使用 from agentkit.apps import AgentkitSimpleApp 导入 AgentKit 应用
* 在代码中初始化 App：使用 app = AgentkitSimpleApp() 初始化应用
* 装饰调用函数：使用 @app.entrypoint 装饰器装饰调用函数
* 让 AgentKit Runtime 控制 Agent 运行：使用 app.run() 启动应用
* 从上一步创建的Oauth2资源凭据提供商中检索 Lark 凭据

#### 在 VeADK Agent 中使用托管的凭据访问飞书文档 API

In [ ]:
!mkdir -p test-agents

In [ ]:
%%writefile test-agents/requirements.txt
lark_oapi

In [ ]:
%%writefile test-agents/agent.py

from veadk import Runner, Agent
from agentkit.apps import AgentkitSimpleApp
import json
from typing import AsyncGenerator

import lark_oapi as lark
from lark_oapi.api.docx.v1 import *
from lark_oapi.core.model import RequestOption
from veadk.integrations.ve_identity import (
    AuthRequestProcessor,
    is_pending_auth_event,
    get_function_call_id,
    get_function_call_auth_config,
    VeIdentityFunctionTool,
    oauth2_auth,
)
from google.adk.events.event import Event
from google.adk.utils.context_utils import Aclosing
from google.genai import types


async def lark_document_query(document_id: str, *, access_token: str) -> str:
    """
    查询飞书文档内容

    Args:
        document_id: 飞书文档ID（从文档链接中提取的最后一部分）
        access_token: 飞书 API OAuth2.0 访问令牌

    Returns:
        文档内容的JSON字符串
    """
    try:
        print(f"查询飞书文档: {document_id}")
        print(f"使用访问令牌: {access_token[:8]}...")

        client = (
            lark.Client.builder()
            .enable_set_token(True)
            .log_level(lark.LogLevel.DEBUG)
            .build()
        )
        request: RawContentDocumentRequest = (
            RawContentDocumentRequest.builder().lang(0).document_id(document_id).build()
        )

        response: RawContentDocumentResponse = client.docx.v1.document.raw_content(
            request, RequestOption.builder().user_access_token(access_token).build()
        )

        if not response.success():
            lark.logger.error(
                f"client.docx.v1.document.raw_content failed, code: {response.code}, msg: {response.msg}, log_id: {response.get_log_id()}, resp: \n{json.dumps(json.loads(response.raw.content), indent=4, ensure_ascii=False)}"
            )
            return f"文档查询失败: {response.msg}"

        # 处理业务结果
        return lark.JSON.marshal(response.data, indent=4)

    except Exception as e:
        return f"文档查询出错: {str(e)}"


# 创建使用 OAuth2 认证的飞书文档查询工具
lark_doc_tool = VeIdentityFunctionTool(
    func=lark_document_query,
    auth_config=oauth2_auth(
        provider_name="lark-user-oauth-provider",
        auth_flow="USER_FEDERATION",
    ),
)

agent: Agent = Agent(
    tools=[lark_doc_tool],
    instruction="""您是一个智能飞书文档助手，能够帮助用户查询和分析飞书文档内容。当用户提供飞书文档链接或询问文档相关问题时，您需要：

1. 识别用户消息中的飞书文档链接（格式如：https://feishu.feishu.cn/docx/WtwHdAngzoEU9IxyfhtcYsHCnDe）
2. 提取文档ID（链接最后一部分，如：WtwHdAngzoEU9IxyfhtcYsHCnDe）
3. 使用 lark_document_query 函数获取文档内容
4. 基于文档内容回答用户的问题或提供分析

功能特点：
- 自动识别飞书文档链接并提取文档ID
- 获取完整的文档内容数据
- 基于文档内容进行智能分析和回答
- 支持各种文档相关的查询和讨论

使用示例：
- "帮我看看这个文档：https://feishu.feishu.cn/docx/WtwHdAngzoEU9IxyfhtcYsHCnDe"
- "这个文档的主要内容是什么？"
- "总结一下文档中的要点"
- "根据文档内容，给我一些建议"

请用专业、友好的语气帮助用户理解和分析文档内容，并根据文档信息提供有价值的见解和建议。""",
)
app = AgentkitSimpleApp()
app_name = "simple_streamable_app"
runner = Runner(app_name=app_name, agent=agent)


@app.entrypoint
async def run(payload: dict, headers: dict) -> AsyncGenerator[Event, None]:
    prompt = payload.get("prompt", "")
    user_id = headers.get("user_id", "anonymous")
    session_id = headers.get("session_id", "default")

    processor = AuthRequestProcessor()
    session_service = runner.short_term_memory.session_service  # type: ignore
    session = await session_service.get_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )
    if not session:
        await session_service.create_session(
            app_name=app_name, user_id=user_id, session_id=session_id
        )

    current_message = types.Content(role="user", parts=[types.Part(text=prompt)])

    while True:
        pending_auth = False

        async with Aclosing(
            runner.run_async(
                user_id=user_id,
                session_id=session_id,
                new_message=current_message,
            )
        ) as agen:
            async for event in agen:
                sse_event = event.model_dump_json(exclude_none=True, by_alias=True)
                yield sse_event

                if is_pending_auth_event(event):
                    auth_request_event_id = get_function_call_id(event)
                    auth_config = get_function_call_auth_config(event)
                    current_message = await processor.process_auth_request(
                        auth_request_event_id, auth_config
                    )
                    pending_auth = True
                    break

        if not pending_auth:
            break


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


### 幕后发生了什么？

当您使用 `AgentkitSimpleApp` 时，它会自动：

* 创建一个监听端口 8000 的 HTTP 服务器
* 实现用于处理 Agent 请求的 `/invoke` 端点
* 实现用于健康检查的 `/ping` 端点（对异步 Agent 非常重要）
* 处理适当的内容类型和响应格式
* 根据 AgentKit 标准管理错误处理 
* 在用户发送消息给Agent的时候，会触发向`Lark Openapi`的请求查看对应文档内容

### 将 Agent 部署到 AgentKit Runtime

AgentKit Platform 的 `CreateRuntime` 操作支持全面的配置选项，让您可以指定容器镜像、环境变量和加密设置。您还可以配置协议设置（HTTP、MCP）和授权机制，以控制客户端与 Agent 的通信方式。

注意： 操作最佳实践是将代码打包为容器，结合火山引擎镜像仓库产品，通过 CI/CD 流水线和基础设施即代码（IaC）将镜像推送到容器镜像仓库。

在本教程中，我们将使用 AgentKit Python SDK 轻松打包您的代码并将其部署到 AgentKit Runtime 上。

#### 配置 AgentKit Runtime 部署

接下来，我们将使用 AgentKit SDK 来配置 AgentKit Runtime 部署，包括EntryPoint、用于访问火山方舟模型的API密钥以及运行的Python版本。

在配置步骤中，将根据您的应用程序代码自动生成 agentkit.yaml 配置文件，它用于在本地保存AgentKit Runtime的配置。

**说明** - 在本教程中使用 cloud 部署模式，即在远程构建镜像并自动部署到AgentKit Runtime上

In [ ]:
from pathlib import Path
from agentkit.toolkit.sdk.config import AgentConfig

current_dir = os.getcwd()
# 如果当前目录不是 test-agents，则切换到该目录
if not current_dir.endswith("test-agents"):
    os.chdir(os.path.join(current_dir, "test-agents"))

agent_file = Path("agentkit.yaml")
if not agent_file.exists():
    agent_file.write_text("")
    print(f"Created empty file: {agent_file}")
else:
    print(f"File already exists, skipping: {agent_file}")

# 指定配置文件路径
config = AgentConfig("agentkit.yaml") \
    .set("common.agent_name", "veidentity_outbound_agent_3lo_lark") \
    .set("common.entry_point", "agent.py") \
    .set("common.language", "Python") \
    .set("common.language_version", "3.12") \
    .set("common.launch_type", "cloud") \
    .set("launch_types.cloud.region", "cn-beijing") \
    .set("launch_types.cloud.runtime_id", "") \
    .set("launch_types.cloud.runtime_name", "") \
    .set("launch_types.cloud.runtime_role_name", "") \
    .set("launch_types.cloud.runtime_envs.MODEL_AGENT_API_KEY", MODEL_API_KEY) \
    .save()

print("Configuration created and saved!")

#### 检查 AgentKit 配置

In [ ]:
!cat agentkit.yaml

#### 启动 AgentKit Runtime

现在我们有了配置文件，让我们将 Agent 启动到 AgentKit Runtime。这将会：

🔨 构建阶段 (Build Phase)
- 生成 Dockerfile - 根据项目配置自动生成容器化配置
- 构建 Docker 镜像 - 将你的 Agent 代码打包成容器镜像
- 准备镜像仓库 - 创建火山引擎 Container Registry (CR) 实例和命名空间
- 推送镜像 - 将构建好的镜像上传到云端镜像仓库

🚀 部署阶段 (Deploy Phase)
- 创建 IAM 角色 - 为 Runtime 生成必要的权限角色
- 创建 Runtime 实例 - 在火山引擎上创建新的 AgentKit Runtime
- 配置认证 - 设置 API Key 或 JWT 认证方式
- 等待就绪 - 监控 Runtime 状态直到达到 Ready 状态
- 获取访问端点 - 返回可用的公网访问地址

In [ ]:
launch_result = sdk.launch()
launch_result

#### 调用 AgentKit Runtime

最后，我们可以使用agent调用我们的 AgentKit Runtime。

##### OAuth 3LO 授权流程说明

在调用过程中会发生以下事情：

1. **首次调用（需要授权）**：
   - 当某个 `user_id` 首次调用 Agent 时，由于用户尚未完成 Lark OAuth 授权，响应中会返回一个 `auth_uri`（授权链接）
   - 用户需要 **点击该链接** 完成飞书账号授权
   - 授权成功后，Agent Identity 会自动保存该用户的访问令牌

2. **后续调用（自动认证）**：
   - 同一 `user_id` 的后续请求会自动使用已缓存的凭证
   - 无需再次点击授权链接，Agent 可以直接访问用户的飞书文档

3. **重新触发授权**：
   - 如需重新授权（例如切换飞书账号），只需修改 `user_id` 为新的值
   - 例如：`user_id="test_user"` → `user_id="test_user_2"`

**提示**：在下面的代码中，您可以通过修改 `user_id` 参数来模拟不同用户的授权流程。


In [ ]:
from __future__ import annotations

import httpx
from google.adk.events.event import Event
from pydantic import BaseModel, ValidationError
from veadk.integrations.ve_identity.utils import (
    get_function_call_auth_config,
    is_pending_auth_event,
)

class StreamingError(BaseModel):
    error: str
    error_type: str
    message: str


def run_sse_client(
    base_url: str,
    user_id: str,
    message: str,
    session_id: str = None,
    api_key: str = None,
):
    """Run SSE client and print events."""

    headers = {
        "Content-Type": "application/json",
        "user_id": user_id,
        "session_id": session_id,
        "Authorization": f"Bearer {api_key}",
    }

    with httpx.Client(timeout=300, headers=headers) as client:
        url = f"{base_url}/invoke"
        body = {"prompt": message}
        sse_headers = {**headers, "Accept": "text/event-stream"}
        with client.stream("POST", url, json=body, headers=sse_headers) as response:
            buffer = ""
            for chunk in response.iter_bytes():
                try:
                    text = chunk.decode("utf-8")
                    buffer += text
                except UnicodeDecodeError as e:
                    continue

                # Parse SSE events
                while "\n\n" in buffer:
                    event, buffer = buffer.split("\n\n", 1)
                    if event.startswith("data: "):
                        data = event[6:]
                        try:
                            # Try StreamingError first
                            try:
                                parsed = StreamingError.model_validate_json(data)
                                print(f"[ERROR] {parsed.message}")
                                break
                            except ValidationError:
                                # If not StreamingError, try Event
                                parsed = Event.model_validate_json(data)

                                if is_pending_auth_event(parsed):
                                    auth_config = get_function_call_auth_config(parsed)
                                    auth_uri = auth_config.exchanged_auth_credential.oauth2.auth_uri
                                    print(f"[AUTH] Please visit {auth_uri} to authenticate")
                                for part in parsed.content.parts:
                                    if part.text:
                                        return parsed
                        except ValidationError:
                            print(f"[EVENT RAW] {data}")
                    else:
                        print(f"[OTHER] {event}")


invoke_response = run_sse_client(
        base_url=launch_result.deploy_result.endpoint_url,
        user_id="test_user",
        session_id="test_session",
        api_key=launch_result.deploy_result.metadata["runtime_apikey"],
        message="帮我看看这个文档：https://feishu.feishu.cn/docx/WtwHdAngzoEU9IxyfhtcYsHCnDe",
    )
invoke_response

#### 处理调用结果

我们现在可以处理调用结果以将其包含在应用程序中

In [ ]:
from IPython.display import Markdown, display
display(Markdown(invoke_response.content.parts[0].text))

#### 清理（可选）

- 现在让我们清理创建的 AgentKit Runtime

In [ ]:
destroy_result = sdk.destroy()
destroy_result

## 总结

您已成功完成了本教程！您学会了如何：

1. 配置 Lark OAuth2 凭据提供商
2. 创建使用火山方舟模型的 VeADK 代理
3. 实现 3-legged OAuth 流程进行用户认证
4. 将agent部署到 AgentKit Runtime
5. 测试具有 Lark 文档查询功能的完整工作流程

现在您可以扩展此示例以集成其他服务或构建更复杂的代理工作流程。